In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import numpy as np

import re
import random

In [2]:
df_action = pd.read_csv("action.csv", index_col = 0)
df_comedy = pd.read_csv("comedy.csv", index_col = 0)
df_drama = pd.read_csv("drama_no_duplicado.csv", index_col = 0)

In [3]:
primeras_drama_8000_df = df_drama.values.tolist()[0:8000]

In [4]:
# Inicializar navegador
def datos_tomatoes (lista):

    driver = webdriver.Chrome() 

    # URL Google para extraer la información solicitada. 

    url = "https://www.google.com/" 

    # Hacer una solicitud GET a la URL especificada.

    driver.get(url)  

    # Maximizar la ventana

    driver.maximize_window()
    


    sleep(5)

    # Aceptar cookies

    driver.find_element("css selector", "#L2AGLb > div").click()

    # Creamos lista vacía para introducir el resultado de las puntuaciones (tomatometer) de cada pelicula.

    total_puntuacion_tomatoes = []

    # Iniciamos bucle for por cada pelicula de los csv que tenemos accediendo por su indice.

    for indice,c in enumerate(lista): 
        

        sleep(random.randint(3,10))
        #creamos un string para pasarle a Google el nombre y el año de la pelicula extraidos de IMDB
        peticion = f"'Tomatometer de {c[1]} del año {c[2]}'"
        #incluimos el string en la barra de busqueda de Google
        driver.find_element("css selector","#APjFqb").send_keys(peticion, Keys.ENTER)  

        #puntuación (tomatometer)
        
        try:

            #creamos una variable con el nombre de la primera pelicula que aparece en Google
            nombre_peli_google = driver.find_element("css selector","#rso > div:nth-child(1) > div > div > div.kb0PBd.cvP2Ce.jGGQ5e > div > div > span > a > h3").text
            

            #si el nombre es igual al nombre de la pelicula que pedimos la añadimos a la lista
            if nombre_peli_google == c[1]:
                
                puntuacion = driver.find_element("css selector","#rso > div:nth-child(1) > div > div > div:nth-child(3) > div > span:nth-child(2)").text.replace("%","").replace("Valoración: ", "")
                total_puntuacion_tomatoes.append(puntuacion)
                print(puntuacion)
            #si el nombre no coincide lo intentamos con la segunda opcion que aparece en google
            else:
                try:
                    nombre_peli_google2 = driver.find_element("css selector","#rso > div:nth-child(2) > div > div > div.kb0PBd.cvP2Ce.jGGQ5e > div > div > span > a > h3").text

                    if nombre_peli_google2 == c[1]:
                
                        puntuacion = driver.find_element("css selector","#rso > div:nth-child(2) > div > div > div:nth-child(3) > div > span:nth-child(2)").text.replace("%","").replace("Valoración: ", "")
                        total_puntuacion_tomatoes.append(puntuacion)
                        print(puntuacion)
                    else:
                        total_puntuacion_tomatoes.append("No tiene datos")
                        print("No tiene datos")
                except:
                    total_puntuacion_tomatoes.append("No tiene datos")
                    print("No tiene datos")
        except: 

            total_puntuacion_tomatoes.append("No tiene datos")
            print("No tiene datos")
        
        print(indice, c[1], c[2], c[4])
        # Salimos a la pag.principal del navegador
        
        driver.back()

    return total_puntuacion_tomatoes    # Impresion lista total 

In [ ]:
#llamamos a la funcion
primeras_8000_drama_tomatoes = datos_tomatoes(primeras_drama_8000_df)

In [ ]:
#convertimos la lista a dataframe
df_drama_imdb_resto = pd.DataFrame(primeras_8000_drama_tomatoes)

df_drama_imdb_resto

In [11]:
#convertimos el dataframe a csv
df_drama_imdb_resto.to_csv("drama_imdb_resto_pelis.csv")